# Exercise 10 - Custom Resources

**GOAL:** The goal of this exercise is to show how to use custom resources

See the documentation on using Ray with custom resources http://ray.readthedocs.io/en/latest/resources.html#custom-resources.

### Concepts for this Exercise - Using Custom Resources

We've discussed how to specify a task's CPU and GPU requirements, but there are many other kinds of resources. For example, a task may require a dataset, which only lives on a few machines, or it may need to be scheduled on a machine with extra memory. These kinds of requirements can be expressed through the use of custom resources.

Custom resources are most useful in the multi-machine setting. However, this exercise illustrates their usage in the single-machine setting.

Ray can be started with a dictionary of custom resources (mapping resource name to resource quantity) as follows.

```python
ray.init(resources={'CustomResource1': 1, 'CustomResource2': 4})
```

The resource requirements of a remote function or actor can be specified in a similar way.

```python
@ray.remote(resources={'CustomResource2': 1})
def f():
    return 1
```

Even if there are many CPUs on the machine, only 4 copies of `f` can be executed concurrently.

Custom resources give applications a great deal of flexibility. For example, if you wish to control precisely which machine a task gets scheduled on, you can simply start each machine with a different custom resource (e.g., start machine `n` with resource `Custom_n` and then tasks that should be scheduled on machine `n` can require resource `Custom_n`. However, this usage has drawbacks because it makes the code less portable and less resilient to machine failures.

In [1]:
import os
os.system('pip install ray')

0

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import ray
import time

In this exercise, we will start Ray using custom resources.

In [3]:
ray.init(num_cpus=8, resources={'Custom1': 4}, include_webui=False, ignore_reinit_error=True)

2019-05-16 12:35:43,765	WARNING worker.py:1337 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-05-16 12:35:43,769	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-16_12-35-43_131/logs.
2019-05-16 12:35:43,886	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:12014 to respond...
2019-05-16 12:35:44,015	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:49338 to respond...
2019-05-16 12:35:44,019	INFO services.py:804 -- Starting Redis shard with 2.58 GB max memory.
2019-05-16 12:35:44,051	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-16_12-35-43_131/logs.
2019-05-16 12:35:44,054	INFO services.py:1427 -- Starting the Plasma object store with 3.87 GB memory using /dev/shm.


{'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2019-05-16_12-35-43_131/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-16_12-35-43_131/sockets/raylet',
 'redis_address': '172.28.0.2:12014',
 'webui_url': None}

**EXERCISE:** Modify the resource requirements of the remote functions below so that the following hold.
- The number of concurrently executing tasks is at most 8 (note that there are 8 CPUs).
- No more than 4 copies of `g` can execute concurrently (note that there are only 4 "Custom1" resources).
- If 4 `g` tasks are executing, then an additional 4 `f` tasks can execute.

You should only need to use the `Custom1` resource.

In [0]:
@ray.remote(num_cpus = 1)
def f():
    time.sleep(0.1)

@ray.remote(resources={'Custom1': 1})
def g():
    time.sleep(0.1)

If you did the above exercise correctly, the next cell should execute without raising an exception.

In [11]:
start = time.time()
ray.get([f.remote() for _ in range(8)])
duration = time.time() - start 
assert duration >= 0.1 and duration < 0.19, '8 f tasks should be able to execute concurrently.'

start = time.time()
ray.get([f.remote() for _ in range(9)])
duration = time.time() - start 
assert duration >= 0.2 and duration < 0.29, 'f tasks should not be able to execute concurrently.'

start = time.time()
ray.get([g.remote() for _ in range(4)])
duration = time.time() - start 
assert duration >= 0.1 and duration < 0.19, '4 g tasks should be able to execute concurrently.'

start = time.time()
ray.get([g.remote() for _ in range(5)])
duration = time.time() - start 
assert duration >= 0.2 and duration < 0.29, '5 g tasks should not be able to execute concurrently.'

start = time.time()
ray.get([f.remote() for _ in range(4)] + [g.remote() for _ in range(4)])
duration = time.time() - start 
assert duration >= 0.1 and duration < 0.19, '4 f and 4 g tasks should be able to execute concurrently.'

start = time.time()
ray.get([f.remote() for _ in range(5)] + [g.remote() for _ in range(4)])
duration = time.time() - start 
assert duration >= 0.2 and duration < 0.29, '5 f and 4 g tasks should not be able to execute concurrently.'

print('Success!')

Success!
